In [6]:
import pprint
import json
import codecs
    
import pprint

In [26]:
# View data structure: json
# List of dicts that can have dicts or lists as values
# 

with codecs.open ('tutorial/conll16st-en-01-12-16-trial/relations.json', 'r', encoding = 'utf-8') as pdtb_file:
    

    # Assign all relations (a list) to a variable
    relations = [json.loads(x) for x in pdtb_file];
    
# Assign one relation to a variable
example_relation = relations[10]

# Assign part of the list to a variable
example_relations = relations[:4]

# Explore the data structure:
# List
print(type(example_relations))

# Dict
print(type(example_relation))
pprint.pprint(example_relation)

# Assign part of a list element to a variable:
arg1 = example_relation['Arg1']
# Dict
print(type(arg1))
pprint.pprint(arg1)
    
    


<class 'list'>
<class 'dict'>
{'Arg1': {'CharacterSpanList': [[2493, 2517]],
          'RawText': 'and told them to cool it',
          'TokenList': [[2493, 2496, 465, 15, 8],
                        [2497, 2501, 466, 15, 9],
                        [2502, 2506, 467, 15, 10],
                        [2507, 2509, 468, 15, 11],
                        [2510, 2514, 469, 15, 12],
                        [2515, 2517, 470, 15, 13]]},
 'Arg2': {'CharacterSpanList': [[2526, 2552]],
          'RawText': "they're ruining the market",
          'TokenList': [[2526, 2530, 472, 15, 15],
                        [2530, 2533, 473, 15, 16],
                        [2534, 2541, 474, 15, 17],
                        [2542, 2545, 475, 15, 18],
                        [2546, 2552, 476, 15, 19]]},
 'Connective': {'CharacterSpanList': [[2518, 2525]],
                'RawText': 'because',
                'TokenList': [[2518, 2525, 471, 15, 14]]},
 'DocID': 'wsj_1000',
 'ID': 14887,
 'Sense': ['Contingency.Cau

In [85]:
# Open json file containing the parses and assign them to a variable:
with codecs.open('tutorial/conll16st-en-01-12-16-trial/parses.json', 'r', encoding = 'utf8') as parse_file:
    parses = json.load(parse_file)
    
# Explore data structure
print(type(parses))

example_parse = list(parses.items())[0]
print(len(example_parse))

example_parse_value = list(parses.values())
print(len(example_parse_value))
print(type(example_parse_value[0]))
pprint.pprint(example_parse_value)



<class 'dict'>
2
1
<class 'dict'>
[{'sentences': [{'dependencies': [['nn', 'Inc.-4', 'Kemper-1'],
                                  ['nn', 'Inc.-4', 'Financial-2'],
                                  ['nn', 'Inc.-4', 'Services-3'],
                                  ['nsubj', 'cut-16', 'Inc.-4'],
                                  ['vmod', 'cut-16', 'charging-6'],
                                  ['mark', 'ruining-11', 'that-7'],
                                  ['nn', 'trading-9', 'program-8'],
                                  ['nsubj', 'ruining-11', 'trading-9'],
                                  ['aux', 'ruining-11', 'is-10'],
                                  ['ccomp', 'charging-6', 'ruining-11'],
                                  ['det', 'market-14', 'the-12'],
                                  ['nn', 'market-14', 'stock-13'],
                                  ['dobj', 'ruining-11', 'market-14'],
                                  ['root', 'ROOT-0', 'cut-16'],
                     

In [53]:
# Work with example relation (defined above)

# Get doc_id of the example
example_doc_id = example_relation['DocID']
print(example_doc_id)
# Get sentence id of arg1, arg2 and the connective (which is given in the tutorial as 15, but we 
# know it is list index 3 
# in the token list)

example_sentence_id_arg1 = example_relation['Arg1']['TokenList'][0][3]
example_sentence_id_c = example_relation['Connective']['TokenList'][0][3]
example_sentence_id_arg2 = example_relation['Arg2']['TokenList'][0][3]

# The sentence ids could be different (in the case of arg1 in the previous sentence), but here they are the same:
print(example_sentence_id_arg1)
print(example_sentence_id_c)
print(example_sentence_id_arg2)

# Print parse tree of example relations (which happens to be in one sentence):
print(parses[en_doc_id]['sentences'][example_sentence_id_arg1]['parsetree'])

wsj_1000
15
15
15
( (S (NP (PRP We)) (VP (VBP 've) (VP (VP (VBN talked) (PP (TO to) (NP (NP (NNS proponents)) (PP (IN of) (NP (NN index) (NN arbitrage)))))) (CC and) (VP (VBD told) (NP (PRP them)) (S (VP (TO to) (VP (VB cool) (NP (PRP it)) (SBAR (IN because) (S (NP (PRP they)) (VP (VBP 're) (VP (VBG ruining) (NP (DT the) (NN market)))))))))))) (. .)) )



In [105]:
# Get words and POS 

example_sentence = (parses[en_doc_id]['sentences'][example_sentence_id_arg1])
#pprint.pprint(example_sentence['words'])

# Explore data structure
print(type(example_sentence['words']))
print(type(example_sentence['words'][0][1]))
#pprint.pprint(exampe_sentence['words'])


# Print words and their POS:
for word in example_sentence['words']:
    print(word[0], word[1]['PartOfSpeech'])

<class 'list'>
<class 'dict'>
We PRP
've VBP
talked VBN
to TO
proponents NNS
of IN
index NN
arbitrage NN
and CC
told VBD
them PRP
to TO
cool VB
it PRP
because IN
they PRP
're VBP
ruining VBG
the DT
market NN
. .


In [ ]:
# Explore parse tree

from nltk.tree import Tree

example_tree = parses[en_doc_id]['sentences'][example_sentence_id_arg1]['parsetree']

print(example_tree)
print(type(example_tree))

# Load string as tree:

tree_nltk = Tree.fromstring(example_tree)
print(type(tree_nltk))

# Parsing the tree:

#print(tree_nltk.flatten())

# Draw your tree :-) 
tree_nltk.draw() 



( (S (NP (PRP We)) (VP (VBP 've) (VP (VP (VBN talked) (PP (TO to) (NP (NP (NNS proponents)) (PP (IN of) (NP (NN index) (NN arbitrage)))))) (CC and) (VP (VBD told) (NP (PRP them)) (S (VP (TO to) (VP (VB cool) (NP (PRP it)) (SBAR (IN because) (S (NP (PRP they)) (VP (VBP 're) (VP (VBG ruining) (NP (DT the) (NN market)))))))))))) (. .)) )

<class 'str'>
<class 'nltk.tree.Tree'>
